In [3]:
! pip install contractions
! pip install punktab

ERROR: Could not find a version that satisfies the requirement punktab (from versions: none)
ERROR: No matching distribution found for punktab


In [5]:
import pandas as pd
import numpy as np
import contractions
import re
import contractions
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from tqdm import tqdm
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from nltk.util import ngrams

ModuleNotFoundError: No module named 'wordcloud'

In [ ]:
df = pd.read_csv('/Users/jorgejimenez/Documents/UP/NLP/RegresionLogistica/deepsea.csv')
df.head()

In [ ]:
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')
stop_words = stopwords.words('english')
lemmatizer = WordNetLemmatizer()

In [ ]:
mapper = {
    'biology': 0,
    'meme': 1
}
df['etiqueta'] = df['category'].map(mapper)
df.head()

In [ ]:
def procesamiento_de_texto(texto):
  #Arreglar contracciones:
  texto = contractions.fix(texto)
  #Hacer todo minusculo
  texto = texto.lower()
  #Regex
  texto = re.sub(r'[^a-z\s]', '', texto)
  #Tokenizar
  texto = word_tokenize(texto)
  #Quitar stopwords
  texto = [token for token in texto if token not in stop_words]
  #Lemmatizar
  texto = [lemmatizer.lemmatize(word) for word in texto]
  #Juntar todo
  texto = ' '.join(texto)
  # Regresar resultado
  return texto

In [ ]:
tqdm.pandas()

In [ ]:
df['clean_tweet'] =  df['tweet'].progress_apply(procesamiento_de_texto)
df.head()

In [ ]:
def generate_ngrams(n, arreglo):
  x = []
  for texto in arreglo:
    n_grams = ngrams(word_tokenize(texto), n)
    for grams in n_grams:
      resultado = ' '.join(grams)
      x.append(resultado)
  return pd.Series(x).value_counts()

In [ ]:
n = 2
cym_ngrams = generate_ngrams(n, df['clean_tweet']).head(30)
cym_ngrams.head(10)

In [ ]:
data=cym_ngrams
plt.barh(data.index, data.values)
plt.xlabel('Frecuencia')
plt.gca().invert_yaxis()
plt.tight_layout()

In [ ]:
nube=WordCloud(width=2000, height=2000, colormap='Wistia_r', background_color='black',min_font_size=8).generate_from_frequencies(cym_ngrams)

In [ ]:
plt.figure(figsize=(5,5))
plt.imshow(nube)
plt.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
# ngram with meme only
n = 2
cym_ngrams = generate_ngrams(n, df[df['category'] == 'meme']['clean_tweet']).head(30)
cym_ngrams.head(10)

In [ ]:
data=cym_ngrams
plt.barh(data.index, data.values)
plt.xlabel('Frecuencia')
plt.gca().invert_yaxis()
plt.tight_layout()

In [ ]:
nube=WordCloud(width=2000, height=2000, colormap='Wistia_r', background_color='black',min_font_size=8).generate_from_frequencies(cym_ngrams)

In [ ]:
plt.figure(figsize=(5,5))
plt.imshow(nube)
plt.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
 # ngram with biology only
 n = 2
cym_ngrams = generate_ngrams(n, df[df['category'] == 'biology']['clean_tweet']).head(30)
cym_ngrams.head(10)

In [ ]:
nube=WordCloud(width=2000, height=2000, colormap='Wistia_r', background_color='black',min_font_size=8).generate_from_frequencies(cym_ngrams)

In [ ]:
plt.figure(figsize=(5,5))
plt.imshow(nube)
plt.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tdidf_vectorizer = TfidfVectorizer(ngram_range=(1,2))
vectorized_tweets = tdidf_vectorizer.fit_transform(df['clean_tweet'])
vectorized_tweets

In [ ]:
from sklearn.manifold import TSNE
model = TSNE(n_components=2, init='random')
tsne_result = model.fit_transform(vectorized_tweets)
tsne_result

In [ ]:
df['tsne_1'] = tsne_result[:,0]
df['tsne_2'] = tsne_result[:,1]

In [ ]:
import plotly.express as px
fig = px.scatter(
    data_frame=df,
    x=df['tsne_1'],
    y=df['tsne_2'],
    color=df['etiqueta'],
    template='plotly_dark',
    hover_data=['tweet']
)
fig.show()

In [ ]:
from sklearn.model_selection import train_test_split
X = df['clean_tweet']
Y = df['etiqueta']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=0.7, random_state=101)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(ngram_range=(1,2))

#Fit transform en datos de entrenamiento
X_train_vectorized = tfidf.fit_transform(X_train)

#Transform en datos de prueba
X_test_vectorized = tfidf.transform(X_test)

In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(X_train_vectorized, Y_train)

In [ ]:
y_pred = model.predict(X_test_vectorized)

In [ ]:
y_pred

In [ ]:
Y_test

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

In [ ]:
sns.heatmap(confusion_matrix(Y_test,y_pred), annot=True, fmt='.2f')

In [ ]:
print(classification_report(Y_test,y_pred))
# precision me importan los falsos positivos
# recall me importan los falsos negativos
# f1-score me importa el overall de todo